In [4]:
import os, json
import zipfile
import time

In [6]:
# sets printer to be used
if not os.path.exists('params.json'):
    print('Criando arquivos de parâmetros')
    with open('params.json', 'w') as file:
        json.dump({'printer_ip':''}, file)


Criando arquivos de parâmetros


In [ ]:

printerPort = r'\\192.168.0.26\zebraPrinter'

# gets python file path and the path to the files to be printed
pythonFolder = r'c:\Dropbox\Karmapec\automating printing' #os.path.dirname(__file__)

printingFolderName = 'Etiquetas'
printingJobsFolder = os.path.join(pythonFolder,printingFolderName)

# creates folder to store pending jobs
try:
    os.makedirs(printingJobsFolder)
except FileExistsError:
    None

In [3]:
while True:
    pendingJobsFiles = os.listdir(printingJobsFolder)
    if '.txt' in [os.path.splitext(file)[1] for file in pendingJobsFiles]: 
        for file in pendingJobsFiles:
            fileExtension = os.path.splitext(file)[1]
            filePath = os.path.join(printingJobsFolder,file)
            # print(filePath)

            if fileExtension == '.txt':
                printCommand = f'copy "{filePath}" {printerPort}'
                print('printed: ', file)
                # os.system(printCommand)
                os.remove(filePath)
                
            
    else:
        if '.zip' in [os.path.splitext(file)[1] for file in pendingJobsFiles]:
            for file in pendingJobsFiles:
                fileExtension = os.path.splitext(file)[1]
                filePath = os.path.join(printingJobsFolder, file)
                
                if fileExtension == '.zip':
                    with zipfile.ZipFile(filePath,'r') as zip_ref:
                        zip_ref.extractall(printingJobsFolder)
                    print('unziped: ', file)
                    os.remove(filePath)
                    break
    
    time.sleep(1)


printed:  Etiqueta de envio.txt
unziped:  Etiqueta MercadoEnvios-202403221622.zip
printed:  Etiqueta de envio.txt
